### Importing Libraries

In [ ]:
import os
import geopandas as gpd
from PIL import Image, ImageDraw
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from osgeo import gdal
from shapely.geometry import shape, box
import fiona
import rasterio
from rasterio.windows import Window

### Cutting Tif files into 1024x1024 sized tif files

In [ ]:
def is_tile_empty(data):
    # Check if the tile contains any data (non-zero values)
    try:
        return np.all(data[3] == 0)
    except Exception as e:
        return True

In [ ]:
def split_tiff(input_path, output_folder, tile_size):
    try:
        with rasterio.open(input_path) as src:
            width, height = src.width, src.height
            num_tiles_x = (width + tile_size - 1) // tile_size
            num_tiles_y = (height + tile_size - 1) // tile_size

            for tile_y in range(num_tiles_y):
                for tile_x in range(num_tiles_x):
                    left = tile_x * tile_size
                    upper = tile_y * tile_size
                    right = min(left + tile_size, width)
                    lower = min(upper + tile_size, height)

                    window = Window(left, upper, right - left, lower - upper)
                    transform = src.window_transform(window)

                    # Read the data within the window and create a new cropped raster
                    data = src.read(window=window)
                    profile = src.profile.copy()
                    profile.update(width=right - left, height=lower - upper, transform=transform)
                    
                    if is_tile_empty(data):
                        # Skip saving the file if it's empty
                        continue

                    # Extract the original file name without extension
                    file_name = os.path.splitext(os.path.basename(input_path))[0]

                    # Save the cropped raster with metadata
                    tile_path = os.path.join(output_folder, f"{file_name}_{tile_x}_{tile_y}.tif")
                    with rasterio.open(tile_path, 'w', **profile) as dst:
                        dst.write(data)
    except Exception as e:
        print(f"Error processing '{input_path}': {e}")

In [ ]:
def crop_directory(directory_path, output_path, tile_size):

    for location_folder in os.listdir(directory_path):
        location_folder_path = os.path.join(directory_path, location_folder)

        geotiff_folder_path = os.path.join(location_folder_path, "Geotiff")

        output_subfolder = os.path.join("Cropped_Tif_File", location_folder)
        output_folder = os.path.join(output_path, output_subfolder)
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        else:
            continue
        
        if not os.path.exists(geotiff_folder_path):
            print("path does not exists", geotiff_folder_path)
            continue

        for file in os.listdir(geotiff_folder_path):
            if file.endswith(".tif") or file.endswith(".tiff"):
                input_file = os.path.join(geotiff_folder_path, file)
                split_tiff(input_file, output_folder, tile_size)

In [ ]:
home_directory = "Orthos, DSMs & Shapefiles"
output_directory = "Dataset"
tile_size = 1024

crop_directory(home_directory, output_directory, tile_size)

### Converting tif files to png files

In [ ]:
from PIL import Image
import os

def convert_tif_to_png(tif_path, png_path):
    if not os.path.exists(png_path):
        os.makedirs(png_path)

    # Get the list of TIF files in the tif_path directory
    tif_files = [f for f in os.listdir(tif_path) if f.endswith('.tif')]

    for tif_file in tif_files:
        # Construct the full paths for TIF and PNG files
        tif_file_path = os.path.join(tif_path, tif_file)
        png_file_path = os.path.join(png_path, tif_file.replace('.tif', '.png'))

        # Skip the file if the corresponding PNG file already exists
        if os.path.exists(png_file_path):
            continue

        tif_image = Image.open(tif_file_path)

        # Ensure the TIF image has four channels (R, G, B, and an additional channel)
        if tif_image.mode != 'RGBA':
            print(tif_file)
            raise ValueError("The input TIF file should have 4 channels (RGBA).")

        # Separate the channels
        r, g, b, a = tif_image.split()

        # Create a new image with RGB mode (3 channels) and paste the RGB channels from the TIF
        rgb_image = Image.merge("RGB", (r, g, b))

        rgb_image.save(png_file_path)
        tif_image.close()

In [ ]:
convert_tif_to_png("../Dataset/test_files/", "../Dataset/test_png_files/")

In [ ]:
tif_directory = "Dataset/cropped_tif_files"
png_directory = "Dataset/cropped_png_files"

places = os.listdir(tif_directory)

for place in places:
    print(place)
    tif_place_path = os.path.join(tif_directory, place)
    png_place_path = os.path.join(png_directory, place)
    convert_tif_to_png(tif_place_path, png_place_path)

### Removing unnecessary files

In [ ]:
from PIL import Image
import os


In [ ]:
def is_mostly_empty(image_path, threshold):
    image = Image.open(image_path)
    width, height = image.size
    pixel_data = list(image.getdata())

    non_white_pixel_count = 0
    for pixel in pixel_data:
        if pixel != (255, 255, 255):  # White pixel in RGB is (255, 255, 255)
            non_white_pixel_count += 1

    per = non_white_pixel_count/(1024*1024)
    per = per*100;
    # if(per<threshold):
    #     return True
    # return False
    return per

In [ ]:
tif_directory = "Dataset/cropped_tif_files"
png_directory = "../Dataset/cropped_png_files"

places = os.listdir(png_directory)
below_15 = []
below_30 = []
below_20 = []
below_10 = []
for place in places:
    # print(place)
    png_place_path = os.path.join(png_directory, place)
    # Get the list of TIF files in the tif_path directory
    png_files = [f for f in os.listdir(png_place_path) if f.endswith('.png')]

    count = 0
    for png_file in png_files:
        png_file_path = os.path.join(png_place_path, png_file)
        per = is_mostly_empty(png_file_path, 10)
        if per<10:
            count += 1
            below_10.append(png_file_path)
        elif per<15:
            below_15.append(png_file_path)
        elif per<20:
            below_20.append(png_file_path)
        elif per<30:
            below_30.append(png_file_path)
    # print(count)
print(len(below_10))
print(len(below_15))
print(len(below_20))
print(len(below_30))

### Get polygons of a particular tif file

In [ ]:
import rasterio

def get_tif_bounding_box(tif_path):
    with rasterio.open(tif_path) as dataset:
        bounds = dataset.bounds
        min_x, min_y, max_x, max_y = bounds.left, bounds.bottom, bounds.right, bounds.top
        return (min_x, min_y, max_x, max_y)

In [ ]:
# # Function to get the bounding box of the TIF file
# def get_tif_bounding_box(tif_path):
#     # Replace this with the appropriate code to obtain the bounding box from the TIF file.
#     # For example, using GDAL library to read raster metadata.
#     # Here's an example assuming you are using GDAL:
#     ds = gdal.Open(tif_path)
#     ulx, xres, xskew, uly, yskew, yres = ds.GetGeoTransform()
#     lrx = ulx + (ds.RasterXSize * xres)
#     lry = uly + (ds.RasterYSize * yres)
#     return (ulx, uly, lrx, lry)

In [ ]:
def get_polygons_shp_file(shp_path):
    print(shp_path)
    # Open the shapefile and read the polygons:
    polygons = []
    with fiona.open(shp_path, "r") as shapefile:
        for feature in shapefile:
            geometry = shape(feature["geometry"])
            polygons.append((geometry, feature["properties"]))
    return polygons

In [ ]:
def get_filtered_polygons(polygons, tif_path):
    # Get the bounding box of the TIF file
    tif_bbox = get_tif_bounding_box(tif_path)
    
    # Filter the polygons based on the intersection with the TIF file's bounding box:
    filtered_polygons = []
    for polygon, properties in polygons:
        polygon_bbox = polygon.bounds
        if box(*tif_bbox).intersects(box(*polygon_bbox)):
            filtered_polygons.append(polygon)

    return filtered_polygons

In [ ]:
import fiona 
from shapely.geometry import shape, box

shp_path = "../Dataset/annotations/Chigicherla_Builtup_Area.shp"
tif_path = "../Dataset/cropped_tif_files/Chigicherla/Chigichrla_Ortho_H1_2_10.tif"

polygons = get_polygons_shp_file(shp_path)
filtered_polygons = get_filtered_polygons(polygons, tif_path)
print(len(filtered_polygons))

### Create masks

In [ ]:
import rasterio
# code to convert CRS coordinate to pixel coordinate
dataset = rasterio.open("../Dataset/cropped_tif_files/Chigicherla/Chigichrla_Ortho_H1_2_10.tif")
height = dataset.height
width = dataset.width
transform = dataset.transform

In [ ]:
def fill_between(polygon):
    """
    Returns: a bool array
    """
    img = Image.new('1', (width, height), False)
    ImageDraw.Draw(img).polygon(polygon, outline=True, fill=True)
    mask = np.array(img)

    return mask

In [ ]:
import numpy as np
from PIL import Image, ImageDraw

# Step 1: Initialize masks with the shape (height, width, number_of_polygons)
masks = np.zeros((height, width, len(filtered_polygons)), dtype=np.uint8)

# Step 2, 3, and 4: Loop through the filtered polygons, generate masks, and stack them
for idx, polygon in enumerate(filtered_polygons):
    coordinates = list()
    for point in polygon.exterior.coords:
        x, y = point
        pixel_x, pixel_y = ~transform * (x, y)
        pixel_x = width - 1 if pixel_x > width else pixel_x
        pixel_y = height - 1 if pixel_y > height else pixel_y
        coordinates.append((pixel_x, pixel_y))
    
    mask = fill_between(coordinates)
    masks[:, :, idx] = mask

# Step 5: Create a single image with different colors for each mask
unique_colors = np.random.randint(0, 255, (len(filtered_polygons), 3), dtype=np.uint8)
output_image = np.zeros((height, width, 3), dtype=np.uint8)

for idx, color in enumerate(unique_colors):
    mask_idx = np.where(masks[:, :, idx])
    output_image[mask_idx] = color

# Save the output image
output_image_pil = Image.fromarray(output_image)
output_image_pil.save('combined_masks.png')